In [1]:
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import date, datetime
from multiprocessing.spawn import import_main_path
from keras.preprocessing.sequence import TimeseriesGenerator
from pandas_datareader import data
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np

# GET THE DATA

In [2]:
today = date.today()
# print(today)
start_date = '2010-01-01'
end_date = '2022-06-27'

panel_data_antm = data.DataReader('ANTM.JK','yahoo',start_date, end_date)
panel_data_asii = data.DataReader('ASII.JK','yahoo',start_date, end_date)
panel_data_icbp = data.DataReader('ICBP.JK','yahoo',start_date, end_date)
panel_data_jsmr = data.DataReader('JSMR.JK','yahoo',start_date, end_date)


# ADDING NEW VARIABEL

In [3]:
#antm
panel_data_antm['average'] = panel_data_antm[['Open','Close','High','Low']].mean(axis=1).round(2)
panel_data_antm['daily_return'] = panel_data_antm['Open'] - panel_data_antm['Close']
#asii
panel_data_asii['average'] = panel_data_asii[['Open','Close','High','Low']].mean(axis=1).round(2)
panel_data_asii['daily_return'] = panel_data_asii['Open'] - panel_data_asii['Close']
#icbp
panel_data_icbp['average'] = panel_data_icbp[['Open','Close','High','Low']].mean(axis=1).round(2)
panel_data_icbp['daily_return'] = panel_data_icbp['Open'] - panel_data_icbp['Close']
#jsmr
panel_data_jsmr['average'] = panel_data_jsmr[['Open','Close','High','Low']].mean(axis=1).round(2)
panel_data_jsmr['daily_return'] = panel_data_jsmr['Open'] - panel_data_jsmr['Close']

In [4]:
df_antm = pd.DataFrame(panel_data_antm)
df_asii = pd.DataFrame(panel_data_asii)
df_icbp = pd.DataFrame(panel_data_icbp)
df_jsmr = pd.DataFrame(panel_data_jsmr)

In [5]:
# apply a min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_antm = pd.DataFrame(scaler.fit_transform(panel_data_antm),columns=['high','low','open','close','volume','adj_close','average','daily_return'])
df_asii = pd.DataFrame(scaler.fit_transform(panel_data_asii),columns=['high','low','open','close','volume','adj_close','average','daily_return'])
df_icbp = pd.DataFrame(scaler.fit_transform(panel_data_icbp),columns=['high','low','open','close','volume','adj_close','average','daily_return'])
df_jsmr = pd.DataFrame(scaler.fit_transform(panel_data_jsmr),columns=['high','low','open','close','volume','adj_close','average','daily_return'])


In [6]:
df_antm.drop(columns=['adj_close','volume'], inplace=True)
df_asii.drop(columns=['adj_close','volume'], inplace=True)
df_icbp.drop(columns=['adj_close','volume'], inplace=True)
df_jsmr.drop(columns=['adj_close','volume'], inplace=True)

In [7]:
# convert to windowed data sets
ylist_antm = panel_data_antm['Adj Close']
ylist_asii = panel_data_asii['Adj Close']
ylist_icbp = panel_data_icbp['Adj Close']
ylist_jsmr = panel_data_jsmr['Adj Close']


In [8]:
#LAGS and PERIOD
n_future = 20
n_past = 3*20
total_period = 4*20

# ANTM DATA

In [9]:
idx_end_antm = len(ylist_antm)
idx_start_antm = idx_end_antm - total_period

X_new_antm = []
y_new_antm = []

while idx_start_antm > 0:
  x_line_antm = ylist_antm[idx_start_antm:idx_start_antm+n_past]
  y_line_antm = ylist_antm[idx_start_antm+n_past:idx_start_antm+total_period]

  X_new_antm.append(x_line_antm)
  y_new_antm.append(y_line_antm)

  idx_start_antm = idx_start_antm - 1

X_new_antm = np.array(X_new_antm)
y_new_antm = np.array(y_new_antm)

# train test split
from sklearn.model_selection import train_test_split
X_train_antm, X_test_antm, y_train_antm, y_test_antm = train_test_split(X_new_antm, y_new_antm, test_size=0.33, random_state=42)

# reshape data into the right format for RNNs
n_samples = X_train_antm.shape[0]
n_timesteps = X_train_antm.shape[1]
n_steps = y_train_antm.shape[1]
n_features = 1

X_train_rs_antm = X_train_antm.reshape(n_samples, n_timesteps, n_features )
X_test_rs_antm = X_test_antm.reshape(X_test_antm.shape[0], n_timesteps, n_features )


# ASII DATA

In [10]:
idx_end_asii = len(ylist_asii)
idx_start_asii = idx_end_asii - total_period

X_new_asii = []
y_new_asii = []

while idx_start_asii > 0:
  x_line_asii = ylist_asii[idx_start_asii:idx_start_asii+n_past]
  y_line_asii = ylist_asii[idx_start_asii+n_past:idx_start_asii+total_period]

  X_new_asii.append(x_line_asii)
  y_new_asii.append(y_line_asii)

  idx_start_asii = idx_start_asii - 1

X_new_asii = np.array(X_new_asii)
y_new_asii = np.array(y_new_asii)

# train test split
from sklearn.model_selection import train_test_split
X_train_asii, X_test_asii, y_train_asii, y_test_asii = train_test_split(X_new_asii, y_new_asii, test_size=0.33, random_state=42)

# reshape data into the right format for RNNs
n_samples = X_train_asii.shape[0]
n_timesteps = X_train_asii.shape[1]
n_steps = y_train_asii.shape[1]
n_features = 1

X_train_rs_asii = X_train_asii.reshape(n_samples, n_timesteps, n_features )
X_test_rs_asii = X_test_asii.reshape(X_test_asii.shape[0], n_timesteps, n_features )


# ICBP DATA

In [11]:
idx_end_icbp = len(ylist_icbp)
idx_start_icbp = idx_end_icbp - total_period

X_new_icbp = []
y_new_icbp = []

while idx_start_icbp > 0:
  x_line_icbp = ylist_icbp[idx_start_icbp:idx_start_icbp+n_past]
  y_line_icbp = ylist_icbp[idx_start_icbp+n_past:idx_start_icbp+total_period]

  X_new_icbp.append(x_line_icbp)
  y_new_icbp.append(y_line_icbp)

  idx_start_icbp = idx_start_icbp - 1

X_new_icbp = np.array(X_new_icbp)
y_new_icbp = np.array(y_new_icbp)

# train test split
from sklearn.model_selection import train_test_split
X_train_icbp, X_test_icbp, y_train_icbp, y_test_icbp = train_test_split(X_new_icbp, y_new_icbp, test_size=0.33, random_state=42)

# reshape data into the right format for RNNs
n_samples = X_train_icbp.shape[0]
n_timesteps = X_train_icbp.shape[1]
n_steps = y_train_icbp.shape[1]
n_features = 1

X_train_rs_icbp = X_train_icbp.reshape(n_samples, n_timesteps, n_features )
X_test_rs_icbp = X_test_icbp.reshape(X_test_icbp.shape[0], n_timesteps, n_features )


# JSMR DATA

In [12]:
idx_end_jsmr = len(ylist_jsmr)
idx_start_jsmr = idx_end_jsmr - total_period

X_new_jsmr = []
y_new_jsmr = []

while idx_start_jsmr > 0:
  x_line_jsmr = ylist_jsmr[idx_start_jsmr:idx_start_jsmr+n_past]
  y_line_jsmr = ylist_jsmr[idx_start_jsmr+n_past:idx_start_jsmr+total_period]

  X_new_jsmr.append(x_line_jsmr)
  y_new_jsmr.append(y_line_jsmr)

  idx_start_jsmr = idx_start_jsmr - 1

X_new_jsmr = np.array(X_new_jsmr)
y_new_jsmr = np.array(y_new_jsmr)

# train test split
from sklearn.model_selection import train_test_split
X_train_jsmr, X_test_jsmr, y_train_jsmr, y_test_jsmr = train_test_split(X_new_jsmr, y_new_jsmr, test_size=0.33, random_state=42)

# reshape data into the right format for RNNs
n_samples = X_train_jsmr.shape[0]
n_timesteps = X_train_jsmr.shape[1]
n_steps = y_train_jsmr.shape[1]
n_features = 1

X_train_rs_jsmr = X_train_jsmr.reshape(n_samples, n_timesteps, n_features )
X_test_rs_jsmr = X_test_jsmr.reshape(X_test_jsmr.shape[0], n_timesteps, n_features )


# TUNING PARAMETER

## LAYER

### ANTM

In [137]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32

simple_model_one_antm = Sequential([
  LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features)),
  Dense(y_train_antm.shape[1]),
])

simple_model_one_antm.summary()

simple_model_one_antm.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_one_antm = simple_model_one_antm.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_one_antm = simple_model_one_antm.predict(X_test_rs_antm)



Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_68 (LSTM)              (None, 8)                 320       
                                                                 
 dense_25 (Dense)            (None, 20)                180       
                                                                 
Total params: 500
Trainable params: 500
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 6s 50ms/step - loss: 1073.5249 - mae: 1073.5249 - val_loss: 1048.5721 - val_mae: 1048.5721
Epoch 2/20
51/51 [==============================] - 1s 26ms/step - loss: 1064.5402 - mae: 1064.5402 - val_loss: 1039.5880 - val_mae: 1039.5880
Epoch 3/20
51/51 [==============================] - 1s 24ms/step - loss: 1055.5559 - mae: 1055.5559 - val_loss: 1030.6040 - val_mae: 1030.6040
Epoch 4/20
51/51 [====

In [138]:
random.seed(42)

simple_model_two_antm = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_two_antm.summary()

simple_model_two_antm.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_two_antm = simple_model_two_antm.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_two_antm = simple_model_two_antm.predict(X_test_rs_antm)

print(r2_score(preds_two_antm, y_test_antm))



Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_69 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_70 (LSTM)              (None, 8)                 544       
                                                                 
 dense_26 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,044
Trainable params: 1,044
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 9s 78ms/step - loss: 1060.8381 - mae: 1060.8381 - mape: 98.0136 - mse: 1441082.3750 - val_loss: 1021.0776 - val_mae: 1021.0776 - val_mape: 95.6859 - val_mse: 1330058.1250
Epoch 2/20
51/51 [==============================] - 3s 50ms/step - loss: 1023.4502 - mae: 10

In [139]:
random.seed(42)

simple_model_three_antm = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm.summary()

simple_model_three_antm.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_three_antm = simple_model_three_antm.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm = simple_model_three_antm.predict(X_test_rs_antm)

print(r2_score(preds_three_antm, y_test_antm))



Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_71 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_72 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_73 (LSTM)              (None, 8)                 544       
                                                                 
 dense_27 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,588
Trainable params: 1,588
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 13s 106ms/step - loss: 1061.4814 - mae: 1061.4814 - mape: 98.0664 - mse: 1442894.8750 - val_loss: 1021.5974 - val_mae: 1021.5974 - 

In [140]:
random.seed(42)

simple_model_four_antm = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_four_antm.summary()

simple_model_four_antm.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_four_antm = simple_model_four_antm.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_four_antm = simple_model_four_antm.predict(X_test_rs_antm)

print(r2_score(preds_four_antm, y_test_antm))



Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_74 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_75 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_76 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_77 (LSTM)              (None, 8)                 544       
                                                                 
 dense_28 (Dense)            (None, 20)                180       
                                                                 
Total params: 2,132
Trainable params: 2,132
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] -

In [141]:
random.seed(42)

simple_model_five_antm = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_five_antm.summary()

simple_model_five_antm.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_five_antm = simple_model_five_antm.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_five_antm = simple_model_five_antm.predict(X_test_rs_antm)

print(r2_score(preds_five_antm, y_test_antm))



Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_78 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_79 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_80 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_81 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_82 (LSTM)              (None, 8)                 544       
                                                                 
 dense_29 (Dense)            (None, 20)                180       
                                                                 
Total params: 2,676
Trainable params: 2,676
Non-train

### ASII

In [142]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_one_asii = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features)),
  Dense(y_train_asii.shape[1]),
])

simple_model_one_asii.summary()

simple_model_one_asii.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=['mae','mape','mse'],
)

smod_history_one_asii = simple_model_one_asii.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_one_asii = simple_model_one_asii.predict(X_test_rs_asii)

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_83 (LSTM)              (None, 8)                 320       
                                                                 
 dense_30 (Dense)            (None, 20)                180       
                                                                 
Total params: 500
Trainable params: 500
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 8s 69ms/step - loss: 5652.0640 - mae: 5652.0640 - mape: 99.9546 - mse: 32996480.0000 - val_loss: 5604.5552 - val_mae: 5604.5552 - val_mape: 99.9051 - val_mse: 32566204.0000
Epoch 2/20
51/51 [==============================] - 1s 28ms/step - loss: 5646.9639 - mae: 5646.9639 - mape: 99.8609 - mse: 32938872.0000 - val_loss: 5599.4556 - val_mae: 5599.4556 - val_mape: 99.8102 - val_mse: 32509064.0000
E

In [143]:
random.seed(42)

simple_model_two_asii = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_two_asii.summary()

simple_model_two_asii.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_two_asii = simple_model_two_asii.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_two_asii = simple_model_two_asii.predict(X_test_rs_asii)

print(r2_score(preds_two_asii, y_test_asii))



Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_84 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_85 (LSTM)              (None, 8)                 544       
                                                                 
 dense_31 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,044
Trainable params: 1,044
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 23s 75ms/step - loss: 5636.9951 - mae: 5636.9951 - mape: 99.6777 - mse: 32826670.0000 - val_loss: 5569.6533 - val_mae: 5569.6533 - val_mape: 99.2557 - val_mse: 32176210.0000
Epoch 2/20
51/51 [==============================] - 2s 44ms/step - loss: 5593.5039 - mae:

In [144]:
random.seed(42)

simple_model_three_asii = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii.summary()

simple_model_three_asii.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_three_asii = simple_model_three_asii.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii = simple_model_three_asii.predict(X_test_rs_asii)

print(r2_score(preds_three_asii, y_test_asii))



Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_86 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_87 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_88 (LSTM)              (None, 8)                 544       
                                                                 
 dense_32 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,588
Trainable params: 1,588
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 20s 157ms/step - loss: 5634.4629 - mae: 5634.4629 - mape: 99.6294 - mse: 32798874.0000 - val_loss: 5568.0884 - val_mae: 5568.0884 -

In [145]:
random.seed(42)

simple_model_four_asii = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_four_asii.summary()

simple_model_four_asii.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_four_asii = simple_model_four_asii.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_four_asii = simple_model_four_asii.predict(X_test_rs_asii)

print(r2_score(preds_four_asii, y_test_asii))



Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_89 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_90 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_91 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_92 (LSTM)              (None, 8)                 544       
                                                                 
 dense_33 (Dense)            (None, 20)                180       
                                                                 
Total params: 2,132
Trainable params: 2,132
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] -

In [146]:
random.seed(42)

simple_model_five_asii = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_five_asii.summary()

simple_model_five_asii.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_five_asii = simple_model_five_asii.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_five_asii = simple_model_five_asii.predict(X_test_rs_asii)

print(r2_score(preds_five_asii, y_test_asii))



Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_93 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_94 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_95 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_96 (LSTM)              (None, 60, 8)             544       
                                                                 
 lstm_97 (LSTM)              (None, 8)                 544       
                                                                 
 dense_34 (Dense)            (None, 20)                180       
                                                                 
Total params: 2,676
Trainable params: 2,676
Non-train

### ICBP

In [147]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_one_icbp = Sequential([
  LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features)),
  Dense(y_train_icbp.shape[1]),
])

simple_model_one_icbp.summary()

simple_model_one_icbp.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_one_icbp = simple_model_one_icbp.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_one_icbp = simple_model_one_icbp.predict(X_test_rs_icbp)

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_98 (LSTM)              (None, 8)                 320       
                                                                 
 dense_35 (Dense)            (None, 20)                180       
                                                                 
Total params: 500
Trainable params: 500
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 7s 44ms/step - loss: 6213.6372 - mae: 6213.6372 - val_loss: 6077.3403 - val_mae: 6077.3403
Epoch 2/20
51/51 [==============================] - 1s 26ms/step - loss: 6204.6543 - mae: 6204.6543 - val_loss: 6068.3560 - val_mae: 6068.3560
Epoch 3/20
51/51 [==============================] - 2s 30ms/step - loss: 6195.6704 - mae: 6195.6704 - val_loss: 6059.3721 - val_mae: 6059.3721
Epoch 4/20
51/51 [====

In [148]:
random.seed(42)

simple_model_two_icbp = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_two_icbp.summary()

simple_model_two_icbp.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_two_icbp = simple_model_two_icbp.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_two_icbp = simple_model_two_icbp.predict(X_test_rs_icbp)

print(r2_score(preds_two_icbp, y_test_icbp))



Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_99 (LSTM)              (None, 60, 8)             320       
                                                                 
 lstm_100 (LSTM)             (None, 8)                 544       
                                                                 
 dense_36 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,044
Trainable params: 1,044
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 10s 95ms/step - loss: 6203.3936 - mae: 6203.3936 - mape: 99.6694 - mse: 46227556.0000 - val_loss: 6055.0278 - val_mae: 6055.0278 - val_mape: 99.2783 - val_mse: 44784720.0000
Epoch 2/20
51/51 [==============================] - 3s 53ms/step - loss: 6168.0176 - mae:

In [149]:
random.seed(42)

simple_model_three_icbp = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp.summary()

simple_model_three_icbp.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_three_icbp = simple_model_three_icbp.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp = simple_model_three_icbp.predict(X_test_rs_icbp)

print(r2_score(preds_three_icbp, y_test_icbp))



Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_101 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_102 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_103 (LSTM)             (None, 8)                 544       
                                                                 
 dense_37 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,588
Trainable params: 1,588
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 15s 128ms/step - loss: 6200.5962 - mae: 6200.5962 - mape: 99.6196 - mse: 46189976.0000 - val_loss: 6050.0029 - val_mae: 6050.0029 -

In [150]:
random.seed(42)

simple_model_four_icbp = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_four_icbp.summary()

simple_model_four_icbp.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_four_icbp = simple_model_four_icbp.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_four_icbp = simple_model_four_icbp.predict(X_test_rs_icbp)

print(r2_score(preds_four_icbp, y_test_icbp))



Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_104 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_105 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_106 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_107 (LSTM)             (None, 8)                 544       
                                                                 
 dense_38 (Dense)            (None, 20)                180       
                                                                 
Total params: 2,132
Trainable params: 2,132
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] -

In [151]:
random.seed(42)

simple_model_five_icbp = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_five_icbp.summary()

simple_model_five_icbp.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_five_icbp = simple_model_five_icbp.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_five_icbp = simple_model_five_icbp.predict(X_test_rs_icbp)

print(r2_score(preds_five_icbp, y_test_icbp))



Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_108 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_109 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_110 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_111 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_112 (LSTM)             (None, 8)                 544       
                                                                 
 dense_39 (Dense)            (None, 20)                180       
                                                                 
Total params: 2,676
Trainable params: 2,676
Non-train

### JSMR

In [152]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_one_jsmr = Sequential([
  LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features)),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_one_jsmr.summary()

simple_model_one_jsmr.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_one_jsmr = simple_model_one_jsmr.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_one_jsmr = simple_model_one_jsmr.predict(X_test_rs_jsmr)

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_113 (LSTM)             (None, 8)                 320       
                                                                 
 dense_40 (Dense)            (None, 20)                180       
                                                                 
Total params: 500
Trainable params: 500
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 5s 45ms/step - loss: 4614.8340 - mae: 4614.8340 - val_loss: 4610.2661 - val_mae: 4610.2661
Epoch 2/20
51/51 [==============================] - 1s 25ms/step - loss: 4598.1274 - mae: 4598.1274 - val_loss: 4593.5146 - val_mae: 4593.5146
Epoch 3/20
51/51 [==============================] - 1s 24ms/step - loss: 4581.3755 - mae: 4581.3755 - val_loss: 4576.7622 - val_mae: 4576.7622
Epoch 4/20
51/51 [====

In [153]:
random.seed(42)

simple_model_two_jsmr = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_two_jsmr.summary()

simple_model_two_jsmr.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_two_jsmr = simple_model_two_jsmr.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_two_jsmr = simple_model_two_jsmr.predict(X_test_rs_jsmr)

print(r2_score(preds_two_jsmr, y_test_jsmr))



Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_114 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_115 (LSTM)             (None, 8)                 544       
                                                                 
 dense_41 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,044
Trainable params: 1,044
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 12s 87ms/step - loss: 4607.5991 - mae: 4607.5991 - mape: 99.6536 - mse: 22332920.0000 - val_loss: 4595.4917 - val_mae: 4595.4917 - val_mape: 99.2723 - val_mse: 22365930.0000
Epoch 2/20
51/51 [==============================] - 2s 48ms/step - loss: 4576.4346 - mae:

In [154]:
random.seed(42)

simple_model_three_jsmr = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr.summary()

simple_model_three_jsmr.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_three_jsmr = simple_model_three_jsmr.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr = simple_model_three_jsmr.predict(X_test_rs_jsmr)

print(r2_score(preds_three_jsmr, y_test_jsmr))



Model: "sequential_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_116 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_117 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_118 (LSTM)             (None, 8)                 544       
                                                                 
 dense_42 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,588
Trainable params: 1,588
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 23s 161ms/step - loss: 4602.5352 - mae: 4602.5352 - mape: 99.5356 - mse: 22286404.0000 - val_loss: 4581.9194 - val_mae: 4581.9194 -

In [155]:
random.seed(42)

simple_model_four_jsmr = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_four_jsmr.summary()

simple_model_four_jsmr.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_four_jsmr = simple_model_four_jsmr.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_four_jsmr = simple_model_four_jsmr.predict(X_test_rs_jsmr)

print(r2_score(preds_four_jsmr, y_test_jsmr))



Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_119 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_120 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_121 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_122 (LSTM)             (None, 8)                 544       
                                                                 
 dense_43 (Dense)            (None, 20)                180       
                                                                 
Total params: 2,132
Trainable params: 2,132
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] -

In [156]:
random.seed(42)

simple_model_five_jsmr = Sequential([
   LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features), return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh', return_sequences=True),
    LSTM(8, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_five_jsmr.summary()

simple_model_five_jsmr.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
 metrics=['mae','mape','mse'],
)

smod_history_five_jsmr = simple_model_five_jsmr.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_five_jsmr = simple_model_five_jsmr.predict(X_test_rs_jsmr)


Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_123 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_124 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_125 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_126 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_127 (LSTM)             (None, 8)                 544       
                                                                 
 dense_44 (Dense)            (None, 20)                180       
                                                                 
Total params: 2,676
Trainable params: 2,676
Non-train

### MATRIKS EVALUASI

#### MAE

In [160]:

# Forecast
# mae score
print('1 layer')
print("mae score antm: "+str(mean_absolute_error(preds_one_antm, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_one_asii, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_one_icbp, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_one_jsmr, y_test_jsmr).round(2)))

print('2 layer')
print("mae score antm: "+str(mean_absolute_error(preds_two_antm, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_two_asii, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_two_icbp, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_two_jsmr, y_test_jsmr).round(2)))

print('3 layer')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_three_asii, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr, y_test_jsmr).round(2)))

print('4 layer')
print("mae score antm: "+str(mean_absolute_error(preds_four_antm, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_four_asii, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_four_icbp, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_four_jsmr, y_test_jsmr).round(2)))

print('5 layer')
print("mae score antm: "+str(mean_absolute_error(preds_five_antm, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_five_asii, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_five_icbp, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_five_jsmr, y_test_jsmr).round(2)))



1 layer
mae score antm: 923.03
mae score asii: 5523.32
mae score icbp: 6091.78
mae score jsmr: 4256.87
2 layer
mae score antm: 478.19
mae score asii: 4801.8
mae score icbp: 5547.64
mae score jsmr: 3977.51
3 layer
mae score antm: 420.88
mae score asii: 4713.58
mae score icbp: 5554.57
mae score jsmr: 3668.13
4 layer
mae score antm: 441.47
mae score asii: 4806.41
mae score icbp: 5656.07
mae score jsmr: 3763.08
5 layer
mae score antm: 442.58
mae score asii: 4704.14
mae score icbp: 5445.67
mae score jsmr: 3667.88


#### MAPE

In [158]:
from sklearn.metrics import mean_absolute_percentage_error
# Forecast
# mape score
print('1 layer')
print("mape score antm: "+str(mean_absolute_percentage_error(preds_one_antm, y_test_antm).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_one_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_one_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_one_jsmr, y_test_jsmr).round(2)))

print('2 layer')
print("mape score antm: "+str(mean_absolute_percentage_error(preds_two_antm, y_test_antm).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_two_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_two_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_two_jsmr, y_test_jsmr).round(2)))

print('3 layer')
print("mape score antm: "+str(mean_absolute_percentage_error(preds_three_antm, y_test_antm).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_three_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr, y_test_jsmr).round(2)))

print('4 layer')
print("mape score antm: "+str(mean_absolute_percentage_error(preds_four_antm, y_test_antm).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_four_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_four_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_four_jsmr, y_test_jsmr).round(2)))

print('5 layer')
print("mape score antm: "+str(mean_absolute_percentage_error(preds_five_antm, y_test_antm).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_five_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_five_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_five_jsmr, y_test_jsmr).round(2)))



1 layer
mape score antm: 5.14
mape score asii: 54.15
mape score icbp: 33.9
mape score jsmr: 12.72
2 layer
mape score antm: 0.71
mape score asii: 5.83
mape score icbp: 7.66
mape score jsmr: 6.48
3 layer
mape score antm: 0.53
mape score asii: 5.17
mape score icbp: 7.75
mape score jsmr: 3.97
4 layer
mape score antm: 0.6
mape score asii: 5.87
mape score icbp: 9.19
mape score jsmr: 4.54
5 layer
mape score antm: 0.6
mape score asii: 5.11
mape score icbp: 6.59
mape score jsmr: 3.97


#### R2

In [159]:
from sklearn.metrics import r2_score
# Forecast
# mape score
print('1 layer')
print("mape score antm: "+str(r2_score(preds_one_antm, y_test_antm).round(2)))
print("mape score asii: "+str(r2_score(preds_one_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(r2_score(preds_one_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(r2_score(preds_one_jsmr, y_test_jsmr).round(2)))

print('2 layer')
print("mape score antm: "+str(r2_score(preds_two_antm, y_test_antm).round(2)))
print("mape score asii: "+str(r2_score(preds_two_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(r2_score(preds_two_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(r2_score(preds_two_jsmr, y_test_jsmr).round(2)))

print('3 layer')
print("mape score antm: "+str(r2_score(preds_three_antm, y_test_antm).round(2)))
print("mape score asii: "+str(r2_score(preds_three_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(r2_score(preds_three_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(r2_score(preds_three_jsmr, y_test_jsmr).round(2)))

print('4 layer')
print("mape score antm: "+str(r2_score(preds_four_antm, y_test_antm).round(2)))
print("mape score asii: "+str(r2_score(preds_four_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(r2_score(preds_four_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(r2_score(preds_four_jsmr, y_test_jsmr).round(2)))

print('5 layer')
print("mape score antm: "+str(r2_score(preds_five_antm, y_test_antm).round(2)))
print("mape score asii: "+str(r2_score(preds_five_asii, y_test_asii).round(2)))
print("mape score icbp: "+str(r2_score(preds_five_icbp, y_test_icbp).round(2)))
print("mape score jsmr: "+str(r2_score(preds_five_jsmr, y_test_jsmr).round(2)))



1 layer
mape score antm: -4155615855676.74
mape score asii: -71765318041729.22
mape score icbp: -9998158765638230.0
mape score jsmr: -495837858575104.75
2 layer
mape score antm: -326172289768.84
mape score asii: -2610888312568.86
mape score icbp: -16852658550471.13
mape score jsmr: -286059389005544.94
3 layer
mape score antm: -22539986560.85
mape score asii: -316915073704335.9
mape score icbp: -4191618131683.0
mape score jsmr: -673945668716.68
4 layer
mape score antm: -6465379666379.04
mape score asii: -4146636568567.22
mape score icbp: -542864071097658.8
mape score jsmr: -53758832754979.58
5 layer
mape score antm: -118875513455.75
mape score asii: -21412351604304.24
mape score icbp: -524611357850307.8
mape score jsmr: -721231362191.83


## NEURON

### ANTM

In [163]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit8 = Sequential([
  LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(8, activation='tanh',return_sequences=True),
  LSTM(8, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit8.summary()

simple_model_three_antm_unit8.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit8 = simple_model_three_antm_unit8.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit8 = simple_model_three_antm_unit8.predict(X_test_rs_antm)

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_131 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_132 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_133 (LSTM)             (None, 8)                 544       
                                                                 
 dense_46 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,588
Trainable params: 1,588
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 24s 219ms/step - loss: 1060.2881 - mae: 1060.2881 - val_loss: 1021.1514 - val_mae: 1021.1514
Epoch 2/20
51/51 [====================

In [164]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit16 = Sequential([
  LSTM(16, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(16, activation='tanh',return_sequences=True),
  LSTM(16, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit16.summary()

simple_model_three_antm_unit16.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit16 = simple_model_three_antm_unit16.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit16 = simple_model_three_antm_unit16.predict(X_test_rs_antm)

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_134 (LSTM)             (None, 60, 16)            1152      
                                                                 
 lstm_135 (LSTM)             (None, 60, 16)            2112      
                                                                 
 lstm_136 (LSTM)             (None, 16)                2112      
                                                                 
 dense_47 (Dense)            (None, 20)                340       
                                                                 
Total params: 5,716
Trainable params: 5,716
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 25s 191ms/step - loss: 1057.9419 - mae: 1057.9419 - val_loss: 1016.3122 - val_mae: 1016.3122
Epoch 2/20
51/51 [====================

In [165]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit64 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit64.summary()

simple_model_three_antm_unit64.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit64 = simple_model_three_antm_unit64.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit64 = simple_model_three_antm_unit64.predict(X_test_rs_antm)

Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_137 (LSTM)             (None, 60, 64)            16896     
                                                                 
 lstm_138 (LSTM)             (None, 60, 64)            33024     
                                                                 
 lstm_139 (LSTM)             (None, 64)                33024     
                                                                 
 dense_48 (Dense)            (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 40s 506ms/step - loss: 1001.2787 - mae: 1001.2787 - val_loss: 898.7371 - val_mae: 898.7371
Epoch 2/20
51/51 [====================

### ASII

In [166]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit8 = Sequential([
  LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(8, activation='tanh',return_sequences=True),
  LSTM(8, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit8.summary()

simple_model_three_asii_unit8.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit8 = simple_model_three_asii_unit8.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit8 = simple_model_three_asii_unit8.predict(X_test_rs_asii)

Model: "sequential_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_140 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_141 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_142 (LSTM)             (None, 8)                 544       
                                                                 
 dense_49 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,588
Trainable params: 1,588
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 18s 133ms/step - loss: 5640.9663 - mae: 5640.9663 - val_loss: 5579.6582 - val_mae: 5579.6582
Epoch 2/20
51/51 [====================

In [167]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit16 = Sequential([
  LSTM(16, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(16, activation='tanh',return_sequences=True),
  LSTM(16, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit16.summary()

simple_model_three_asii_unit16.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit16 = simple_model_three_asii_unit16.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit16 = simple_model_three_asii_unit16.predict(X_test_rs_asii)

Model: "sequential_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_143 (LSTM)             (None, 60, 16)            1152      
                                                                 
 lstm_144 (LSTM)             (None, 60, 16)            2112      
                                                                 
 lstm_145 (LSTM)             (None, 16)                2112      
                                                                 
 dense_50 (Dense)            (None, 20)                340       
                                                                 
Total params: 5,716
Trainable params: 5,716
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 49s 122ms/step - loss: 5632.1602 - mae: 5632.1602 - val_loss: 5563.2822 - val_mae: 5563.2822- loss: 5625.9956 - mae: 5625.
Epoch 2/

In [168]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit64 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit64.summary()

simple_model_three_asii_unit64.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit64 = simple_model_three_asii_unit64.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit64 = simple_model_three_asii_unit64.predict(X_test_rs_asii)

Model: "sequential_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_146 (LSTM)             (None, 60, 64)            16896     
                                                                 
 lstm_147 (LSTM)             (None, 60, 64)            33024     
                                                                 
 lstm_148 (LSTM)             (None, 64)                33024     
                                                                 
 dense_51 (Dense)            (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 31s 468ms/step - loss: 5590.3823 - mae: 5590.3823 - val_loss: 5476.5317 - val_mae: 5476.5317
Epoch 2/20
51/51 [==================

### ICBP

In [169]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit8 = Sequential([
  LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(8, activation='tanh',return_sequences=True),
  LSTM(8, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit8.summary()

simple_model_three_icbp_unit8.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit8 = simple_model_three_icbp_unit8.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit8 = simple_model_three_icbp_unit8.predict(X_test_rs_icbp)

Model: "sequential_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_149 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_150 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_151 (LSTM)             (None, 8)                 544       
                                                                 
 dense_52 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,588
Trainable params: 1,588
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 19s 147ms/step - loss: 6215.6421 - mae: 6215.6421 - val_loss: 6081.2319 - val_mae: 6081.2319
Epoch 2/20
51/51 [====================

In [170]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit16 = Sequential([
  LSTM(16, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(16, activation='tanh',return_sequences=True),
  LSTM(16, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit16.summary()

simple_model_three_icbp_unit16.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit16 = simple_model_three_icbp_unit16.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit16 = simple_model_three_icbp_unit16.predict(X_test_rs_icbp)

Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_152 (LSTM)             (None, 60, 16)            1152      
                                                                 
 lstm_153 (LSTM)             (None, 60, 16)            2112      
                                                                 
 lstm_154 (LSTM)             (None, 16)                2112      
                                                                 
 dense_53 (Dense)            (None, 20)                340       
                                                                 
Total params: 5,716
Trainable params: 5,716
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 17s 132ms/step - loss: 6213.7671 - mae: 6213.7671 - val_loss: 6077.0630 - val_mae: 6077.0640
Epoch 2/20
51/51 [====================

In [171]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit64 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit64.summary()

simple_model_three_icbp_unit64.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit64 = simple_model_three_icbp_unit64.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit64 = simple_model_three_icbp_unit64.predict(X_test_rs_icbp)

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_155 (LSTM)             (None, 60, 64)            16896     
                                                                 
 lstm_156 (LSTM)             (None, 60, 64)            33024     
                                                                 
 lstm_157 (LSTM)             (None, 64)                33024     
                                                                 
 dense_54 (Dense)            (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 42s 597ms/step - loss: 6202.2441 - mae: 6202.2441 - val_loss: 6053.9585 - val_mae: 6053.9585
Epoch 2/20
51/51 [==================

### JSMR

In [172]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit8 = Sequential([
  LSTM(8, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(8, activation='tanh',return_sequences=True),
  LSTM(8, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit8.summary()

simple_model_three_jsmr_unit8.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit8 = simple_model_three_jsmr_unit8.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit8 = simple_model_three_jsmr_unit8.predict(X_test_rs_jsmr)

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_158 (LSTM)             (None, 60, 8)             320       
                                                                 
 lstm_159 (LSTM)             (None, 60, 8)             544       
                                                                 
 lstm_160 (LSTM)             (None, 8)                 544       
                                                                 
 dense_55 (Dense)            (None, 20)                180       
                                                                 
Total params: 1,588
Trainable params: 1,588
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 15s 125ms/step - loss: 4620.4800 - mae: 4620.4800 - val_loss: 4621.6914 - val_mae: 4621.6914
Epoch 2/20
51/51 [====================

In [173]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit16 = Sequential([
  LSTM(16, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(16, activation='tanh',return_sequences=True),
  LSTM(16, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit16.summary()

simple_model_three_jsmr_unit16.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit16 = simple_model_three_jsmr_unit16.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit16 = simple_model_three_jsmr_unit16.predict(X_test_rs_jsmr)

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_161 (LSTM)             (None, 60, 16)            1152      
                                                                 
 lstm_162 (LSTM)             (None, 60, 16)            2112      
                                                                 
 lstm_163 (LSTM)             (None, 16)                2112      
                                                                 
 dense_56 (Dense)            (None, 20)                340       
                                                                 
Total params: 5,716
Trainable params: 5,716
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 15s 130ms/step - loss: 4618.1743 - mae: 4618.1743 - val_loss: 4617.1616 - val_mae: 4617.1616
Epoch 2/20
51/51 [====================

In [174]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit64 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit64.summary()

simple_model_three_jsmr_unit64.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit64 = simple_model_three_jsmr_unit64.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit64 = simple_model_three_jsmr_unit64.predict(X_test_rs_jsmr)

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_164 (LSTM)             (None, 60, 64)            16896     
                                                                 
 lstm_165 (LSTM)             (None, 60, 64)            33024     
                                                                 
 lstm_166 (LSTM)             (None, 64)                33024     
                                                                 
 dense_57 (Dense)            (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 55s 613ms/step - loss: 4606.3682 - mae: 4606.3682 - val_loss: 4593.2461 - val_mae: 4593.2461
Epoch 2/20
51/51 [==================

### MATRIKS EVALUASI

#### MAE

In [175]:

# Forecast
# mae score
print('8 node')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit8, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit8, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit8, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit8, y_test_jsmr).round(2)))

print('16 node')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit16, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit16, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit16, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit16, y_test_jsmr).round(2)))

print('64 node')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit64, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit64, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit64, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit64, y_test_jsmr).round(2)))

8 node
mae score antm: 479.43
mae score asii: 4792.09
mae score icbp: 6177.78
mae score jsmr: 4497.06
16 node
mae score antm: 443.0
mae score asii: 4705.34
mae score icbp: 6094.76
mae score jsmr: 4414.7
64 node
mae score antm: 405.61
mae score asii: 2969.8
mae score icbp: 5654.24
mae score jsmr: 3953.82


#### MAPE

In [176]:
from sklearn.metrics import mean_absolute_percentage_error
# Forecast
# mae score
print('8 node')
print("mae score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit8, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit8, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit8, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit8, y_test_jsmr).round(2)))

print('16 node')
print("mae score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit16, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit16, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit16, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit16, y_test_jsmr).round(2)))

print('64 node')
print("mae score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit64, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit64, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit64, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit64, y_test_jsmr).round(2)))


8 node
mae score antm: 0.71
mae score asii: 5.75
mae score icbp: 65.94
mae score jsmr: 47.64
16 node
mae score antm: 0.6
mae score asii: 5.11
mae score icbp: 34.49
mae score jsmr: 24.98
64 node
mae score antm: 0.45
mae score asii: 1.12
mae score icbp: 9.16
mae score jsmr: 6.2


#### R2

In [177]:
from sklearn.metrics import r2_score
# Forecast
# mae score
print('8 node')
print("mae score antm: "+str(r2_score(preds_three_antm_unit8, y_test_antm).round(2)))
print("mae score asii: "+str(r2_score(preds_three_asii_unit8, y_test_asii).round(2)))
print("mae score icbp: "+str(r2_score(preds_three_icbp_unit8, y_test_icbp).round(2)))
print("mae score jsmr: "+str(r2_score(preds_three_jsmr_unit8, y_test_jsmr).round(2)))

print('16 node')
print("mae score antm: "+str(r2_score(preds_three_antm_unit16, y_test_antm).round(2)))
print("mae score asii: "+str(r2_score(preds_three_asii_unit16, y_test_asii).round(2)))
print("mae score icbp: "+str(r2_score(preds_three_icbp_unit16, y_test_icbp).round(2)))
print("mae score jsmr: "+str(r2_score(preds_three_jsmr_unit16, y_test_jsmr).round(2)))

print('64 node')
print("mae score antm: "+str(r2_score(preds_three_antm_unit64, y_test_antm).round(2)))
print("mae score asii: "+str(r2_score(preds_three_asii_unit64, y_test_asii).round(2)))
print("mae score icbp: "+str(r2_score(preds_three_icbp_unit64, y_test_icbp).round(2)))
print("mae score jsmr: "+str(r2_score(preds_three_jsmr_unit64, y_test_jsmr).round(2)))



8 node
mae score antm: -2002656147065.29
mae score asii: -4870050289994.1
mae score icbp: -6.30694189603795e+17
mae score jsmr: -493086524102807.8
16 node
mae score antm: -2388337758647.83
mae score asii: -1477295304297.62
mae score icbp: -306685270903447.44
mae score jsmr: -152447903917046.06
64 node
mae score antm: -52825997034.46
mae score asii: -731499928228.89
mae score icbp: -68004796324191.09
mae score jsmr: -260921490556348.84


## LEARNING RATE

### ANTM

In [13]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit64_lr1 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit64_lr1.summary()

simple_model_three_antm_unit64_lr1.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit64_lr1 = simple_model_three_antm_unit64_lr1.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit64_lr1 = simple_model_three_antm_unit64_lr1.predict(X_test_rs_antm)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 64)            16896     
                                                                 
 lstm_1 (LSTM)               (None, 60, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 17s 152ms/step - loss: 993.7996 - mae: 993.7996 - val_loss: 883.2815 - val_mae: 883.2815
Epoch 2/20
51/51 [=========================

In [14]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit64_lr2 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit64_lr2.summary()

simple_model_three_antm_unit64_lr2.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit64_lr2 = simple_model_three_antm_unit64_lr2.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit64_lr2 = simple_model_three_antm_unit64_lr2.predict(X_test_rs_antm)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_4 (LSTM)               (None, 60, 64)            33024     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 18s 156ms/step - loss: 1061.9766 - mae: 1061.9766 - val_loss: 1024.2603 - val_mae: 1024.2603
Epoch 2/20
51/51 [===================

In [15]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit64_lr3 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit64_lr3.summary()

simple_model_three_antm_unit64_lr3.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit64_lr3 = simple_model_three_antm_unit64_lr3.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit64_lr3 = simple_model_three_antm_unit64_lr3.predict(X_test_rs_antm)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_7 (LSTM)               (None, 60, 64)            33024     
                                                                 
 lstm_8 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 17s 143ms/step - loss: 1075.3306 - mae: 1075.3306 - val_loss: 1052.4879 - val_mae: 1052.4879
Epoch 2/20
51/51 [===================

### ASII

In [16]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit64_lr1 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit64_lr1.summary()

simple_model_three_asii_unit64_lr1.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit64_lr1 = simple_model_three_asii_unit64_lr1.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit64_lr1 = simple_model_three_asii_unit64_lr1.predict(X_test_rs_asii)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_10 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_11 (LSTM)              (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 17s 161ms/step - loss: 5598.0547 - mae: 5598.0547 - val_loss: 5492.1455 - val_mae: 5492.1455
Epoch 2/20
51/51 [===================

In [17]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit64_lr2 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit64_lr2.summary()

simple_model_three_asii_unit64_lr2.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit64_lr2 = simple_model_three_asii_unit64_lr2.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit64_lr2 = simple_model_three_asii_unit64_lr2.predict(X_test_rs_asii)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_13 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_14 (LSTM)              (None, 64)                33024     
                                                                 
 dense_4 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 18s 152ms/step - loss: 5639.0762 - mae: 5639.0762 - val_loss: 5577.5029 - val_mae: 5577.5029
Epoch 2/20
51/51 [===================

In [18]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit64_lr3 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit64_lr3.summary()

simple_model_three_asii_unit64_lr3.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit64_lr3 = simple_model_three_asii_unit64_lr3.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit64_lr3 = simple_model_three_asii_unit64_lr3.predict(X_test_rs_asii)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_16 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_17 (LSTM)              (None, 64)                33024     
                                                                 
 dense_5 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 15s 134ms/step - loss: 5651.9912 - mae: 5651.9912 - val_loss: 5604.5620 - val_mae: 5604.5620
Epoch 2/20
51/51 [===================

### ICBP

In [19]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit64_lr1 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit64_lr1.summary()

simple_model_three_icbp_unit64_lr1.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit64_lr1 = simple_model_three_icbp_unit64_lr1.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit64_lr1 = simple_model_three_icbp_unit64_lr1.predict(X_test_rs_icbp)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_19 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_20 (LSTM)              (None, 64)                33024     
                                                                 
 dense_6 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 14s 138ms/step - loss: 6126.7456 - mae: 6126.7456 - val_loss: 5896.9404 - val_mae: 5896.9404
Epoch 2/20
51/51 [===================

In [20]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit64_lr2 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit64_lr2.summary()

simple_model_three_icbp_unit64_lr2.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit64_lr2 = simple_model_three_icbp_unit64_lr2.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit64_lr2 = simple_model_three_icbp_unit64_lr2.predict(X_test_rs_icbp)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_22 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_23 (LSTM)              (None, 64)                33024     
                                                                 
 dense_7 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 14s 129ms/step - loss: 6202.1787 - mae: 6202.1787 - val_loss: 6053.6128 - val_mae: 6053.6128
Epoch 2/20
51/51 [===================

In [21]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit64_lr3 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit64_lr3.summary()

simple_model_three_icbp_unit64_lr3.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit64_lr3 = simple_model_three_icbp_unit64_lr3.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit64_lr3 = simple_model_three_icbp_unit64_lr3.predict(X_test_rs_icbp)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_25 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_26 (LSTM)              (None, 64)                33024     
                                                                 
 dense_8 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 14s 135ms/step - loss: 6215.7388 - mae: 6215.7388 - val_loss: 6081.5942 - val_mae: 6081.5942
Epoch 2/20
51/51 [===================

### JSMR

In [22]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit64_lr1 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit64_lr1.summary()

simple_model_three_jsmr_unit64_lr1.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit64_lr1 = simple_model_three_jsmr_unit64_lr1.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit64_lr1 = simple_model_three_jsmr_unit64_lr1.predict(X_test_rs_jsmr)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_27 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_28 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_29 (LSTM)              (None, 64)                33024     
                                                                 
 dense_9 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 15s 135ms/step - loss: 4538.8789 - mae: 4538.8789 - val_loss: 4452.6982 - val_mae: 4452.6982
Epoch 2/20
51/51 [===================

In [23]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit64_lr2 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit64_lr2.summary()

simple_model_three_jsmr_unit64_lr2.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit64_lr2 = simple_model_three_jsmr_unit64_lr2.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit64_lr2 = simple_model_three_jsmr_unit64_lr2.predict(X_test_rs_jsmr)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_31 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_32 (LSTM)              (None, 64)                33024     
                                                                 
 dense_10 (Dense)            (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 14s 134ms/step - loss: 4606.2549 - mae: 4606.2549 - val_loss: 4592.5044 - val_mae: 4592.5044
Epoch 2/20
51/51 [==================

In [24]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit64_lr3 = Sequential([
  LSTM(64, activation='tanh',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='tanh',return_sequences=True),
  LSTM(64, activation='tanh'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit64_lr3.summary()

simple_model_three_jsmr_unit64_lr3.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit64_lr3 = simple_model_three_jsmr_unit64_lr3.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit64_lr3 = simple_model_three_jsmr_unit64_lr3.predict(X_test_rs_jsmr)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_33 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_34 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_35 (LSTM)              (None, 64)                33024     
                                                                 
 dense_11 (Dense)            (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 17s 145ms/step - loss: 4620.1279 - mae: 4620.1279 - val_loss: 4621.7363 - val_mae: 4621.7363
Epoch 2/20
51/51 [==================

### MATRIKS EVALUASI

#### MAE

In [25]:
print('lrate 0.1')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit64_lr1, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit64_lr1, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit64_lr1, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit64_lr1, y_test_jsmr).round(2)))
print('lrate 0.01')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit64_lr2, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit64_lr2, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit64_lr2, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit64_lr2, y_test_jsmr).round(2)))
print('lrate 0.001')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit64_lr3, y_test_antm).round(2)))
print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit64_lr3, y_test_asii).round(2)))
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit64_lr3, y_test_icbp).round(2)))
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit64_lr3, y_test_jsmr).round(2)))

lrate 0.1
mae score antm: 405.62
mae score asii: 3276.18
mae score icbp: 3505.95
mae score jsmr: 1422.42
lrate 0.01
mae score antm: 517.81
mae score asii: 5007.88
mae score icbp: 5614.91
mae score jsmr: 3922.26
lrate 0.001
mae score antm: 1032.75
mae score asii: 5555.17
mae score icbp: 6201.6
mae score jsmr: 4521.78


#### MAPE

In [28]:
from sklearn.metrics import mean_absolute_percentage_error
print('lrate 0.1')
print("mape score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit64_lr1, y_test_antm).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit64_lr1, y_test_asii).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit64_lr1, y_test_icbp).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit64_lr1, y_test_jsmr).round(2)))
print('lrate 0.01')
print("mape score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit64_lr2, y_test_antm).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit64_lr2, y_test_asii).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit64_lr2, y_test_icbp).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit64_lr2, y_test_jsmr).round(2)))
print('lrate 0.001')
print("mape score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit64_lr3, y_test_antm).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit64_lr3, y_test_asii).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit64_lr3, y_test_icbp).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit64_lr3, y_test_jsmr).round(2)))

lrate 0.1
mape score antm: 0.45
mape score asii: 1.39
mape score icbp: 1.07
mape score jsmr: 0.43
lrate 0.01
mape score antm: 0.84
mape score asii: 8.11
mape score icbp: 8.55
mape score jsmr: 5.86
lrate 0.001
mape score antm: 14.76
mape score asii: 79.22
mape score icbp: 88.8
mape score jsmr: 64.92


#### R2

In [30]:
print('lrate 0.1')
print("mae score antm: "+str(r2_score(preds_three_antm_unit64_lr1, y_test_antm).round(2)))
print("mae score asii: "+str(r2_score(preds_three_asii_unit64_lr1, y_test_asii).round(2)))
print("mae score icbp: "+str(r2_score(preds_three_icbp_unit64_lr1, y_test_icbp).round(2)))
print("mae score jsmr: "+str(r2_score(preds_three_jsmr_unit64_lr1, y_test_jsmr).round(2)))
print('lrate 0.01')
print("mae score antm: "+str(r2_score(preds_three_antm_unit64_lr2, y_test_antm).round(2)))
print("mae score asii: "+str(r2_score(preds_three_asii_unit64_lr2, y_test_asii).round(2)))
print("mae score icbp: "+str(r2_score(preds_three_icbp_unit64_lr2, y_test_icbp).round(2)))
print("mae score jsmr: "+str(r2_score(preds_three_jsmr_unit64_lr2, y_test_jsmr).round(2)))
print('lrate 0.001')
print("mae score antm: "+str(r2_score(preds_three_antm_unit64_lr3, y_test_antm).round(2)))
print("mae score asii: "+str(r2_score(preds_three_asii_unit64_lr3, y_test_asii).round(2)))
print("mae score icbp: "+str(r2_score(preds_three_icbp_unit64_lr3, y_test_icbp).round(2)))
print("mae score jsmr: "+str(r2_score(preds_three_jsmr_unit64_lr3, y_test_jsmr).round(2)))

lrate 0.1
mae score antm: -580909980602.15
mae score asii: -5124913624098.13
mae score icbp: -3973430169326.98
mae score jsmr: -100232445686.31
lrate 0.01
mae score antm: -1021399312016.93
mae score asii: -15359541608621.65
mae score icbp: -546243635937247.8
mae score jsmr: -3174817612811.9
lrate 0.001
mae score antm: -328291604551660.2
mae score asii: -1588881395750651.2
mae score icbp: -5335910796446037.0
mae score jsmr: -199424585576489.4
